In [1]:
from __future__ import annotations

import asyncio
import nest_asyncio

# make asyncio.run() works in notebook
nest_asyncio.apply()

In [2]:
from metagpt.config2 import Config
from metagpt.provider.llm_provider_registry import create_llm_instance
from metagpt.utils.cost_manager import CostManager

cfg = Config.default()

llm = create_llm_instance(cfg.llm)
# llm.cost_manager = CostManager()
llm

2024-03-17 09:04:26.206 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /root/workspace/StreamChatPlayground/notebooks


In [7]:
asyncio.run(llm.aask("你好"))

Of course, I'm here to help. How can I assist you

2024-03-16 23:03:23.170 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 14, completion_tokens: 18


 today?


"Of course, I'm here to help. How can I assist you today?"

In [9]:
import json
from typing import Union, Literal
from pydantic import Field, model_validator

from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message, Task, TaskResult
from metagpt.strategy.task_type import TaskType

from metagpt.tools.tool_recommend import BM25ToolRecommender, ToolRecommender
from metagpt.actions.di.execute_nb_code import ExecuteNbCode
from metagpt.actions.di.write_analysis_code import CheckData, WriteAnalysisCode
from metagpt.prompts.di.write_analysis_code import DATA_INFO
from metagpt.utils.common import CodeParser

In [10]:
REACT_THINK_PROMPT = """
# User Requirement
{user_requirement}
# Context
{context}

Output a json following the format:
```json
{{
    "thoughts": str = "Thoughts on current situation, reflect on how you should proceed to fulfill the user requirement",
    "state": bool = "Decide whether you need to take more actions to complete the user requirement. Return true if you think so. Return false if you think the requirement has been completely fulfilled."
}}
```
"""

class CodeInterpreter(Role):
    name: str = "OpenDevin"
    profile: str = "CodeInterpreter"
    auto_run: bool = True
    use_plan: bool = True
    use_reflection: bool = False
    execute_code: ExecuteNbCode = Field(default_factory=ExecuteNbCode, exclude=True)
    tools: Union[str, list[str]] = []  # Use special symbol ["<all>"] to indicate use of all registered tools
    tool_recommender: ToolRecommender = None
    react_mode: Literal["plan_and_act", "react"] = "plan_and_act"
    max_react_loop: int = 10  # used for react mode

    @model_validator(mode="after")
    def set_plan_and_tool(self) -> "Interpreter":
        self._set_react_mode(react_mode=self.react_mode, max_react_loop=self.max_react_loop, auto_run=self.auto_run)
        self.use_plan = (
            self.react_mode == "plan_and_act"
        )  # create a flag for convenience, overwrite any passed-in value
        if self.tools:
            self.tool_recommender = BM25ToolRecommender(tools=self.tools)
        self.set_actions([WriteAnalysisCode])
        self._set_state(0)
        return self

    @property
    def working_memory(self):
        return self.rc.working_memory

    async def _think(self) -> bool:
        """Useful in 'react' mode. Use LLM to decide whether and what to do next."""
        user_requirement = self.get_memories()[0].content
        context = self.working_memory.get()

        if not context:
            # just started the run, we need action certainly
            self.working_memory.add(self.get_memories()[0])  # add user requirement to working memory
            self._set_state(0)
            return True

        prompt = REACT_THINK_PROMPT.format(user_requirement=user_requirement, context=context)
        rsp = await self.llm.aask(prompt)
        rsp_dict = json.loads(CodeParser.parse_code(block=None, text=rsp))
        self.working_memory.add(Message(content=rsp_dict["thoughts"], role="assistant"))
        need_action = rsp_dict["state"]
        self._set_state(0) if need_action else self._set_state(-1)

        return need_action

    async def _act(self) -> Message:
        """Useful in 'react' mode. Return a Message conforming to Role._act interface."""
        code, _, _ = await self._write_and_exec_code()
        return Message(content=code, role="assistant", cause_by=WriteAnalysisCode)

    async def _plan_and_act(self) -> Message:
        rsp = await super()._plan_and_act()
        await self.execute_code.terminate()
        return rsp

    async def _act_on_task(self, current_task: Task) -> TaskResult:
        """Useful in 'plan_and_act' mode. Wrap the output in a TaskResult for review and confirmation."""
        code, result, is_success = await self._write_and_exec_code()
        task_result = TaskResult(code=code, result=result, is_success=is_success)
        return task_result

    async def _write_and_exec_code(self, max_retry: int = 3):
        counter = 0
        success = False

        # plan info
        plan_status = self.planner.get_plan_status() if self.use_plan else ""

        # tool info
        if self.tools:
            context = (
                self.working_memory.get()[-1].content if self.working_memory.get() else ""
            )  # thoughts from _think stage in 'react' mode
            plan = self.planner.plan if self.use_plan else None
            tool_info = await self.tool_recommender.get_recommended_tool_info(context=context, plan=plan)
        else:
            tool_info = ""

        # data info
        await self._check_data()

        while not success and counter < max_retry:
            ### write code ###
            code, cause_by = await self._write_code(counter, plan_status, tool_info)

            self.working_memory.add(Message(content=code, role="assistant", cause_by=cause_by))

            ### execute code ###
            result, success = await self.execute_code.run(code)
            print(result)

            self.working_memory.add(Message(content=result, role="user", cause_by=ExecuteNbCode))

            ### process execution result ###
            counter += 1

            if not success and counter >= max_retry:
                logger.info("coding failed!")
                counter = 0  # redo the task again with help of human suggestions
                # review, _ = await self.planner.ask_review(auto_run=False, trigger=ReviewConst.CODE_REVIEW_TRIGGER)
                # if ReviewConst.CHANGE_WORDS[0] in review:
                #     counter = 0  # redo the task again with help of human suggestions

        return code, result, success

    async def _write_code(
        self,
        counter: int,
        plan_status: str = "",
        tool_info: str = "",
    ):
        todo = self.rc.todo  # todo is WriteAnalysisCode
        logger.info(f"ready to {todo.name}")
        use_reflection = counter > 0 and self.use_reflection  # only use reflection after the first trial

        user_requirement = self.get_memories()[0].content

        code = await todo.run(
            user_requirement=user_requirement,
            plan_status=plan_status,
            tool_info=tool_info,
            working_memory=self.working_memory.get(),
            use_reflection=use_reflection,
        )

        return code, todo

    async def _check_data(self):
        if (
            not self.use_plan
            or not self.planner.plan.get_finished_tasks()
            or self.planner.plan.current_task.task_type
            not in [
                TaskType.DATA_PREPROCESS.type_name,
                TaskType.FEATURE_ENGINEERING.type_name,
                TaskType.MODEL_TRAIN.type_name,
            ]
        ):
            return
        logger.info("Check updated data")
        code = await CheckData().run(self.planner.plan)
        if not code.strip():
            return
        result, success = await self.execute_code.run(code)
        if success:
            print(result)
            data_info = DATA_INFO.format(info=result)
            self.working_memory.add(Message(content=data_info, role="user", cause_by=CheckData))


In [11]:
url = "https://pitchhub.36kr.com/financing-flash"
domain = "快讯"

prompt = f"""抓取{url}中'{domain}'的内容，并整理成markdown存档。

大概的流程：
- 使用工具抓取网页中的可见文本
- 提取网页文本中'{domain}'相关的内容。注意网页中可能包含导航，只需要抽取'{domain}'的具体内容
- 对抽取结果进行归类，并保存成markdown表格: {domain}.md
"""

ci = CodeInterpreter(use_tools=True, tools=["scrape_web_playwright"])
asyncio.run(ci.run(prompt))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.327 seconds.
Prefix dict has been built successfully.


To achieve the goal of scraping the '快讯' content from the provided webpage and organizing it into a Markdown file, the following plan is proposed:

```json
[
    {
        "task_id": "1",
        "dependent_task_ids": [],
        "instruction": "Scrape visible text from the webpage",
        "task_type": "web scraping"
    },
    {
        "task_id": "2",
        "dependent_task_ids": ["1"],
        "instruction": "Extract content related to '快讯'",
        "task_type": "data preprocessing"
    },
    {
        "task_id": "3",
        "dependent_task_ids": ["2"],
        "instruction": "Categorize and format the extracted content into a Markdown table",
        "task_type": "other"
    },
    {
        "task_id": "4",
        "dependent_task_ids": ["3"],
        "instruction": "Save the formatted content into a file named '快讯.md'",
        "task_type": "other"
    }
]
```

This plan outlines a sequence of tasks starting with web scraping, followed by data preprocessing to extract releva

2024-03-17 09:10:03.164 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.010 | Max budget: $10.000 | Current cost: $0.010, prompt_tokens: 474, completion_tokens: 255
2024-03-17 09:10:03.166 | INFO     | metagpt.roles.role:_plan_and_act:492 - ready to take on task task_id='1' dependent_task_ids=[] instruction='Scrape visible text from the webpage' task_type='web scraping' code='' result='' is_success=False is_finished=False
2024-03-17 09:10:03.167 | INFO     | metagpt.tools.tool_recommend:recall_tools:195 - Recalled tools: 
['scrape_web_playwright']; Scores: [-4.028245058449738]


 it to a file.


2024-03-17 09:10:04.315 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.003 | Max budget: $10.000 | Current cost: $0.003, prompt_tokens: 172, completion_tokens: 13
2024-03-17 09:10:04.316 | INFO     | metagpt.tools.tool_recommend:recommend_tools:101 - Recommended tools: 
['scrape_web_playwright']
2024-03-17 09:10:04.317 | INFO     | __main__:_write_code:131 - ready to WriteAnalysisCode


...


```json
["scrape_web_playwright"]
```

The 'scrape_web_playwright' tool can be used to scrape and save the content from a web page, which can then be formatted and saved into a file named '快讯.md'. Since the requirement is to save formatted content, this tool can potentially handle the scraping part, and additional steps may be needed for formatting the content as per

2024-03-17 09:11:59.218 | INFO     | metagpt.utils.cost_manager:update_cost:57 - Total running cost: $0.004 | Max budget: $10.000 | Current cost: $0.004, prompt_tokens: 177, completion_tokens: 88
2024-03-17 09:11:59.219 | INFO     | metagpt.tools.tool_recommend:recommend_tools:101 - Recommended tools: 
['scrape_web_playwright']
2024-03-17 09:11:59.220 | INFO     | __main__:_write_code:131 - ready to WriteAnalysisCode


 the user's requirement.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f79340fcd30>


KeyboardInterrupt: 